## PROGRAM extrct
Extracts data from the N-body output files Single.dat (fort.83) and Binary.dat (fort.82) relating to the overall cluster evolution, ie. half-mass radius and relaxation time, cluster mass, membership, core radius. 

Note: planets are ignored in this version. 

Output in extrct.dat: 

*   Number of single stars + binaries 
*   Number of binaries 
*   Time (Myr) 
*   Relaxation time (Myr) 
*   Total cluster mass (Msun) 
*   Core radius - as determined by Nbody code (pc) 
*   (work in progress) Mass in core (Msun) 
*   (work in progress) Mass outside the tidal radius (Msun) 
*   (work in progress) Maximum stellar distance from cluster centre of mass (pc) 
*   (work in progress) Half-mass radius (pc) 
*   (work in progress) Radius containing inner 10% of cluster mass (pc) 
*   (work in progress) Number of systems (stars + binaries) inside the half-mass radius   
*   (work in progress) Number of systems within 1pc of the cluster centre    
*   (work in progress) Number of systems within the inner lagrangian radius (10%)  
*   (work in progress) Number of systems within the core radius 
*   (work in progress) Velocity dispersion (km/s)

In [1]:
#Necessary libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt
import sys

## Reformatting and reading data from fort.83 and fort.82
(Removing format of fort.83 and fort.82 - run this just once)

In [9]:
import os.path

file83 = os.path.isfile('fort.83-awk')
file82 = os.path.isfile('fort.82-awk')

if not file83:
    !awk '{print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12,$13}' fort.83 > fort.83-awk
    print "Removing format of fort.83"
if not file82:
    !awk '{print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12,$13,$14,$15,$16,$17,$18,$19,$20,$21,$22,$23,$24}' fort.82 > fort.82-awk
    print "Removing format of fort.82"

In [267]:
Ns = 0
Nb = 0
mout = 0
#Output file
output = open('extrct.dat','w')
#Input files
with open("fort.83-awk", 'r') as file1:
    with open("fort.82-awk", 'r') as file2:
        while (file1 and file2):
            rdd = 0
            line11 = file1.readline()
            line21 = file2.readline()
            if not (line11 and line21):
                break
            words11 = line11.split()
            words21 = line21.split()
#Number of single and binary systems            
            Ns = int(words11[0])
            Nb = int(words21[0])
            if Ns != -1000:
                Nsingle = np.array(Ns)
                #Time in physical units
                tphys = np.array(float(words11[1]))
                line12 = file1.readline()
                words12 = line12.split()
                line13 = file1.readline()
                words13 = line13.split()
                #Time block for single systems
                blockS = np.fromfile(file1, sep=' ', count=13*Ns).reshape([Ns,13])
                rd = np.sqrt((blockS[:,5]-float(words12[4]))**2+(blockS[:,6]-float(words12[5]))**2+(blockS[:,7]-float(words12[6]))**2).reshape([Ns,1])
                aux = np.append(blockS, rd, 1)
                #Time block sort by radii
                aux1 = np.array(sorted(aux,key=lambda x: x[13]))
            if Nb != -1000:
                Nbinary = np.array(Nb)
                #Time block for binary systems
                blockB = np.fromfile(file2, sep=' ', count=24*Nb).reshape([Nb,24])
                rdB = np.sqrt((blockB[:,14]-float(words12[4]))**2+(blockB[:,15]-float(words12[5]))**2+(blockB[:,16]-float(words12[6]))**2).reshape([Nb,1])
                aux2 = np.append(blockB, rdB, 1)
                #Time block sort by radii
                aux3 = np.array(sorted(aux2,key=lambda x: x[24]))
                #Total number of systems and binary systems
                data = np.append(Nsingle+Nbinary,Nbinary)
                #Total mass
                mtot = sum(blockS[:,2])+sum(blockB[:,8])+sum(blockB[:,9])
                #Core radius
                rc = np.array(float(words12[1]))
                mc = 0
                m50 = 0
                for x in range(Ns):
                    if aux1[x,13] > float(words12[3]): mout = mout + aux1[x,2]
                for xx in range(Nb):
                    if aux3[xx,24] > float(words12[3]): mout = mout + (aux3[xx,8] + aux3[xx,9])
                for y in range(Ns):
                    if m50 < 0.5*(mtot-mout): 
                        m50 = m50 + aux1[y,2]
                        j = y
                    if aux1[y,13] <= rc: mc = mc + aux1[y,2]               
                trh1 = data[0]/np.log10(0.4*data[0])
                r50 = aux1[j,13]*float(words12[2])
                #Relaxation time
                trh = 0.858*trh1*np.sqrt(r50**3/mtot)
                #Append values and output
                data = np.append(data,tphys)
                data = np.append(data,trh)
                data = np.append(data,mtot)
                data = np.append(data,rc)
                data = np.column_stack(data)
                np.savetxt(output,data,delimiter=" ",fmt="%i %i %1.3f %1.3f %1.3f %1.3f")
output.close()
print "pass"

pass
